In [1]:
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

/Users/kdoherty/miniforge3/envs/light-stable/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Clear the dataset cache to ensure fresh download
import shutil
from pathlib import Path

cache_dir = Path.home() / ".cache" / "huggingface" / "datasets" / "mpg-ranch___light-stable-semantics"
if cache_dir.exists():
    shutil.rmtree(cache_dir)
    print(f"Cleared cache: {cache_dir}")
else:
    print("No cache found to clear")

In [ ]:
# Load all three dataset configurations (cache is cleared, so normal loading)
dataset_default = load_dataset("mpg-ranch/light-stable-semantics", "default", split='train')
dataset_dinov2 = load_dataset("mpg-ranch/light-stable-semantics", "dinov2_base", split='train')
dataset_dinov3 = load_dataset("mpg-ranch/light-stable-semantics", "dinov3_sat", split='train')

# Get the same sample across all configs (they have matching idx values)
sample_idx = 6
obs_default = dataset_default[sample_idx]
obs_dinov2 = dataset_dinov2[sample_idx] 
obs_dinov3 = dataset_dinov3[sample_idx]

print(f"Sample idx: {obs_default['idx']}")
print(f"All configs have same idx: {obs_default['idx'] == obs_dinov2['idx'] == obs_dinov3['idx']}")
print()
print("Default config features:", list(obs_default.keys()))
print("DINOv2 config features:", list(obs_dinov2.keys()))
print("DINOv3 config features:", list(obs_dinov3.keys()))

In [ ]:
# Show embedding shapes
print("=== DINOv2 Base (ViT-B/14) ===")
print(f"CLS token shape: {np.array(obs_dinov2['cls_t0']).shape}")
print(f"Patch tokens shape: {np.array(obs_dinov2['patch_t0']).shape}")
print(f"Patch grid: 16×16 = 256 patches")
print()

print("=== DINOv3 Large (ViT-L/16) ===")
print(f"CLS token shape: {np.array(obs_dinov3['cls_t0']).shape}")
print(f"Patch tokens shape: {np.array(obs_dinov3['patch_t0']).shape}")
print(f"Patch grid: 14×14 = 196 patches")

In [ ]:
# Visualize RGB images at three time points
image_time_points = ['image_t0', 'image_t1', 'image_t2']
images = [obs_default[i] for i in image_time_points]
times = ['Morning (10:00 AM)', 'Noon (12:00 PM)', 'Afternoon (3:00 PM)']

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, (ax, image, time) in enumerate(zip(axes, images, times)):
    ax.imshow(image)
    ax.set_title(time, fontsize=12)
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Visualize DINOv2 patch features (16x16 grid)
patch_t0_v2 = np.array(obs_dinov2['patch_t0']).reshape(16, 16, 768, order='C')
patch_t1_v2 = np.array(obs_dinov2['patch_t1']).reshape(16, 16, 768, order='C') 
patch_t2_v2 = np.array(obs_dinov2['patch_t2']).reshape(16, 16, 768, order='C')

fig, axes = plt.subplots(3, 3, figsize=(12, 12))
fig.suptitle('DINOv2 Base - First 3 Feature Dimensions', fontsize=16)

patches_v2 = [patch_t0_v2, patch_t1_v2, patch_t2_v2]
times = ['Morning (t0)', 'Noon (t1)', 'Afternoon (t2)']

for feature_idx in range(3):  # First 3 features
    for time_idx, (patch, time_label) in enumerate(zip(patches_v2, times)):
        ax = axes[feature_idx, time_idx]
        
        # Extract the feature map for this feature dimension
        feature_map = patch[:, :, feature_idx]
        
        # Display with viridis colormap
        im = ax.imshow(feature_map, cmap='viridis', aspect='equal')
        
        # Add labels
        if feature_idx == 0:
            ax.set_title(time_label, fontsize=12)
        if time_idx == 0:
            ax.set_ylabel(f'Feature {feature_idx}', fontsize=12)
        
        ax.set_xticks([])
        ax.set_yticks([])

plt.tight_layout()
plt.show()

In [ ]:
# Visualize DINOv3 patch features (14x14 grid)
patch_t0_v3 = np.array(obs_dinov3['patch_t0']).reshape(14, 14, 1024, order='C')
patch_t1_v3 = np.array(obs_dinov3['patch_t1']).reshape(14, 14, 1024, order='C') 
patch_t2_v3 = np.array(obs_dinov3['patch_t2']).reshape(14, 14, 1024, order='C')

fig, axes = plt.subplots(3, 3, figsize=(12, 12))
fig.suptitle('DINOv3 Large - First 3 Feature Dimensions', fontsize=16)

patches_v3 = [patch_t0_v3, patch_t1_v3, patch_t2_v3]
times = ['Morning (t0)', 'Noon (t1)', 'Afternoon (t2)']

for feature_idx in range(3):  # First 3 features
    for time_idx, (patch, time_label) in enumerate(zip(patches_v3, times)):
        ax = axes[feature_idx, time_idx]
        
        # Extract the feature map for this feature dimension
        feature_map = patch[:, :, feature_idx]
        
        # Display with viridis colormap
        im = ax.imshow(feature_map, cmap='viridis', aspect='equal')
        
        # Add labels
        if feature_idx == 0:
            ax.set_title(time_label, fontsize=12)
        if time_idx == 0:
            ax.set_ylabel(f'Feature {feature_idx}', fontsize=12)
        
        ax.set_xticks([])
        ax.set_yticks([])

plt.tight_layout()
plt.show()

In [ ]:
# Visualize RGB and Canopy Height from default config
rgb_t2 = np.array(obs_default['image_t2'])
chm = np.array(obs_default['canopy_height'], dtype=np.float32) / 100  # Convert cm to meters

height, width = rgb_t2.shape[:2]
extent = [0, width, height, 0]

# Create figure with constrained layout to handle colorbar properly
fig, axes = plt.subplots(1, 2, figsize=(12, 6), constrained_layout=True)

# RGB plot
axes[0].imshow(rgb_t2, extent=extent)
axes[0].set_title('RGB (3:00 PM)')
axes[0].set_aspect('equal')
axes[0].axis('off')

# CHM plot
cmap = plt.cm.viridis.copy()
cmap.set_bad('white', alpha=0.0)
im_chm = axes[1].imshow(chm, cmap=cmap, extent=extent, interpolation='nearest')
axes[1].set_title('Canopy Height (m)')
axes[1].set_aspect('equal')
axes[1].axis('off')

# Add colorbar without affecting subplot sizes
cbar = fig.colorbar(im_chm, ax=axes[1], shrink=0.8)
cbar.set_label('Plant height (m)')

plt.show()

In [ ]:
# Compare CLS token embeddings between models
cls_dinov2_t1 = np.array(obs_dinov2['cls_t1'])  # 768-dim
cls_dinov3_t1 = np.array(obs_dinov3['cls_t1'])  # 1024-dim

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# DINOv2 CLS token
axes[0].plot(cls_dinov2_t1, 'b-', alpha=0.7, linewidth=1)
axes[0].set_title('DINOv2 Base CLS Token (768-dim)', fontsize=12)
axes[0].set_xlabel('Feature dimension')
axes[0].set_ylabel('Activation value')
axes[0].grid(True, alpha=0.3)

# DINOv3 CLS token  
axes[1].plot(cls_dinov3_t1, 'r-', alpha=0.7, linewidth=1)
axes[1].set_title('DINOv3 Large CLS Token (1024-dim)', fontsize=12)
axes[1].set_xlabel('Feature dimension')
axes[1].set_ylabel('Activation value')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"DINOv2 CLS stats: mean={cls_dinov2_t1.mean():.3f}, std={cls_dinov2_t1.std():.3f}")
print(f"DINOv3 CLS stats: mean={cls_dinov3_t1.mean():.3f}, std={cls_dinov3_t1.std():.3f}")